In [1]:
//master node code
import numpy as np
from PIL import Image

class MasterNode:
    def __init__(self, num_worker_nodes):
        self.num_worker_nodes = num_worker_nodes
        self.comm = CommunicationProtocol()  # Choose a communication protocol here
    
    def process_image(self, input_image_path):
        # Load the input image
        input_image = Image.open(input_image_path)
        width, height = input_image.size
        
        # Divide the input image into smaller chunks
        chunk_width = width // self.num_worker_nodes
        chunk_height = height
        input_chunks = []
        for i in range(self.num_worker_nodes):
            chunk = input_image.crop((i * chunk_width, 0, (i + 1) * chunk_width, chunk_height))
            input_chunks.append(chunk)
        
        # Distribute the chunks to the worker nodes
        for i, chunk in enumerate(input_chunks):
            self.comm.send(i, chunk)
        
        # Collect the processed chunks from the worker nodes
        output_chunks = []
        for i in range(self.num_worker_nodes):
            chunk = self.comm.receive(i)
            output_chunks.append(chunk)
        
        # Reassemble the output chunks into a single image
        output_image = Image.new('RGB', (width, height))
        for i, chunk in enumerate(output_chunks):
            output_image.paste(chunk, (i * chunk_width, 0))
        
        return output_image

In [2]:
//worker node code
import numpy as np
from PIL import Image
from skimage.filters import sobel

class WorkerNode:
    def __init__(self):
        self.comm = CommunicationProtocol()  # Choose a communication protocol here
    
    def process_chunk(self):
        # Receive a chunk of the input image from the master node
        chunk = self.comm.receive()
        
        # Convert the image to grayscale and apply the Sobel edge detection filter
        img_array = np.array(chunk)
        img_array = np.mean(img_array, axis=2)  # Convert to grayscale
        img_array = sobel(img_array)
        
        # Convert the image back to a PIL Image object and return it
        processed_chunk = Image.fromarray(img_array)
        return processed_chunk

In [3]:
// client code
import sys
from PIL import Image

if __name__ == '__main__':
    if len(sys.argv) != 3:
        print('Usage: python client.py <input_image> <num_worker_nodes>')
        sys.exit(1)
    
    input_image_path = sys.argv[1]
    num_worker_nodes = int(sys.argv[2])
    
    master_node = MasterNode(num_worker_nodes)
    output_image = master_node.process_image(input_image_path)
    output_image.show()

ValueError: invalid literal for int() with base 10: 'C:\\Users\\kaush\\AppData\\Roaming\\jupyter\\runtime\\kernel-257ca195-2372-4a82-b135-63502fc25866.json'

In [4]:
class CommunicationProtocol:
    def __init__(self):
        self.worker_nodes = {}  # Dictionary to store the addresses of the worker nodes
    
    def connect(self, worker_id, address):
        """
        Connect to a worker node with the given ID and address.
        """
        self.worker_nodes[worker_id] = address
    
    def send(self, worker_id, data):
        """
        Send data to the worker node with the given ID.
        """
        address = self.worker_nodes[worker_id]
        # Use a socket or other communication mechanism to send the data to the specified address
    
    def receive(self, worker_id):
        """
        Receive data from the worker node with the given ID.
        """
        address = self.worker_nodes[worker_id]
        # Use a socket or other communication mechanism to receive data from the specified address